In [1]:
from IPython.display import display, HTML

def show_df(df):
    display(df.shape)
    display(df.head())

In [2]:
import pandas as pd
raw_dataset = pd.read_csv("FinalProject1_dataset.csv")
show_df(raw_dataset)
display(raw_dataset["direction"].value_counts())

raw_dataset = raw_dataset[raw_dataset.direction != 'UNKNOWN']
#raw_dataset.direction["FROM_INSIDE_TO_OUTSIDE"] = 0
#raw_dataset.direction["FROM_OUTSIDE_TO_INSIDE"] = 1
raw_dataset = raw_dataset.replace("FROM_INSIDE_TO_OUTSIDE", 0)
raw_dataset = raw_dataset.replace("FROM_OUTSIDE_TO_INSIDE", 1)
display(raw_dataset["direction"].value_counts())

(9984, 7)

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367500637,-79,-90
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367501323,-81,-90
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367502416,-83,-91
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367504813,-80,-91
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367504813,-80,-91


FROM_INSIDE_TO_OUTSIDE    4540
FROM_OUTSIDE_TO_INSIDE    4178
UNKNOWN                   1266
Name: direction, dtype: int64

0    4540
1    4178
Name: direction, dtype: int64

In [3]:
clean_dataset = raw_dataset.drop(["timestamp", "_id"], axis=1)
show_df(clean_dataset)

(8718, 5)

,name,locationStatus,direction,rssiOne,rssiTwo
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,-79,-90
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,-81,-90
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,-83,-91
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,-80,-91
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,-80,-91


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

x = clean_dataset.drop(["locationStatus", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo
2294,0,-65,-84
662,0,-88,-93
2521,0,-85,-93
6288,0,-78,-80
9549,1,-84,-78


In [5]:
import matplotlib.pyplot as plt

raw_dataset["rssiOne"]
plt.plot(raw_dataset["timestamp"], raw_dataset["rssiOne"])

# OneR

In [6]:
import pandas as pd
class OneR(object):
    
    def __init__(self):
        self.ideal_variable = None
        self.max_accuracy = 0
    
    def fit(self, X, y):
        response = list()
        result = dict()
        
        dfx = pd.DataFrame(X)
        
        for i in dfx:
            result[str(i)] = dict()
            options_values = set(dfx[i])
            join_data = pd.DataFrame({"variable":dfx[i], "label":y})
            cross_table = pd.crosstab(join_data.variable, join_data.label)
            summary = cross_table.idxmax(axis=1)
            result[str(i)] = dict(summary)
            
            counts = 0
            
            for idx, row in join_data.iterrows():
                if row['label'] == result[str(i)][row['variable']]:
                    counts += 1

            accuracy = (counts/len(y))
            
            if accuracy > self.max_accuracy:
                self.max_accuracy = accuracy
                self.ideal_variable = i

            result_feature = {"variable": str(i), "accuracy":accuracy, "rules": result[str(i)] }  
            response.append(result_feature)
            
        return response

    
    def predict(self, X=None):
        self_ideal_variable = self.ideal_variable + 1
        
    def __repr__(self):
        if self.ideal_variable != None:
            txt = "La mejor variable para tus datos es: " + str(self.ideal_variable)
        else:
            txt = "La mejor variable aun no se ha encontrado, intente ejecutar el metodo fit previamente"
        return txt

In [7]:
clf = OneR()
results = clf.fit(x, y)

print(results)
print(clf)

[{'variable': 'direction', 'accuracy': 0.42991511814636385, 'rules': {0: 'OUTSIDE', 1: 'IN_VESTIBULE'}}, {'variable': 'rssiOne', 'accuracy': 0.5345262674925442, 'rules': {-105: 'INSIDE', -103: 'INSIDE', -102: 'IN_VESTIBULE', -101: 'INSIDE', -100: 'INSIDE', -99: 'INSIDE', -98: 'INSIDE', -97: 'INSIDE', -96: 'INSIDE', -95: 'INSIDE', -94: 'INSIDE', -93: 'INSIDE', -92: 'INSIDE', -91: 'INSIDE', -90: 'INSIDE', -89: 'INSIDE', -88: 'INSIDE', -87: 'INSIDE', -86: 'INSIDE', -85: 'OUTSIDE', -84: 'OUTSIDE', -83: 'IN_VESTIBULE', -82: 'OUTSIDE', -81: 'IN_VESTIBULE', -80: 'IN_VESTIBULE', -79: 'IN_VESTIBULE', -78: 'IN_VESTIBULE', -77: 'IN_VESTIBULE', -76: 'IN_VESTIBULE', -75: 'IN_VESTIBULE', -74: 'IN_VESTIBULE', -73: 'IN_VESTIBULE', -72: 'IN_VESTIBULE', -71: 'IN_VESTIBULE', -70: 'IN_VESTIBULE', -69: 'IN_VESTIBULE', -68: 'OUTSIDE', -67: 'IN_VESTIBULE', -66: 'OUTSIDE', -65: 'IN_VESTIBULE', -64: 'OUTSIDE', -63: 'IN_VESTIBULE', -62: 'IN_VESTIBULE', -61: 'IN_VESTIBULE', -57: 'IN_VESTIBULE'}}, {'variable': 'r

# Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[428 138 169]
 [ 97 730 146]
 [132 139 637]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.58      0.61       735
IN_VESTIBULE       0.72      0.75      0.74       973
     OUTSIDE       0.67      0.70      0.68       908

   micro avg       0.69      0.69      0.69      2616
   macro avg       0.68      0.68      0.68      2616
weighted avg       0.68      0.69      0.68      2616

0.6861620795107034


# Decision Tree (CART)

In [9]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[490 153  92]
 [165 687 121]
 [102 143 663]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.67      0.66       735
IN_VESTIBULE       0.70      0.71      0.70       973
     OUTSIDE       0.76      0.73      0.74       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.70      0.70      0.70      2616

0.7033639143730887


# SVM

In [10]:
from sklearn.svm import SVC
# fit a SVM model to the data
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[480 149 106]
 [101 728 144]
 [ 79  99 730]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.65      0.69       735
IN_VESTIBULE       0.75      0.75      0.75       973
     OUTSIDE       0.74      0.80      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.74      0.74      0.74      2616

0.7408256880733946


# KNN

In [11]:
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[480 173  82]
 [149 682 142]
 [120 121 667]]
              precision    recall  f1-score   support

      INSIDE       0.64      0.65      0.65       735
IN_VESTIBULE       0.70      0.70      0.70       973
     OUTSIDE       0.75      0.73      0.74       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.70      0.70      0.70      2616

0.6991590214067278


# Применяем метод Стандартного отклонения

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train, y_train)
X_train_real_scaled = scaler.transform(X_train)
X_test_real_scaled = scaler.transform(X_test)

C:\anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """
C:\anaconda\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


# Алгоритм Наивного Байеса после Стандартного отклонения

In [13]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train_real_scaled, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[428 138 169]
 [ 97 730 146]
 [132 139 637]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.58      0.61       735
IN_VESTIBULE       0.72      0.75      0.74       973
     OUTSIDE       0.67      0.70      0.68       908

   micro avg       0.69      0.69      0.69      2616
   macro avg       0.68      0.68      0.68      2616
weighted avg       0.68      0.69      0.68      2616

0.6861620795107034


# Алгоритм Деревьев решений после Стандартного отклонения

In [14]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train_real_scaled, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[489 153  93]
 [167 685 121]
 [102 143 663]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.67      0.66       735
IN_VESTIBULE       0.70      0.70      0.70       973
     OUTSIDE       0.76      0.73      0.74       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.70      0.70      0.70      2616

0.702217125382263


# Алгоритм SVM после Стандартного отклонения

In [15]:
svm = SVC()
y_pred = svm.fit(X_train_real_scaled, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[508 154  73]
 [105 715 153]
 [118  83 707]]
              precision    recall  f1-score   support

      INSIDE       0.69      0.69      0.69       735
IN_VESTIBULE       0.75      0.73      0.74       973
     OUTSIDE       0.76      0.78      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.73      0.73      0.73      2616
weighted avg       0.74      0.74      0.74      2616

0.7377675840978594


# Алгоритм KNN после Стандартного отклонения# 

In [16]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train_real_scaled, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[492 161  82]
 [135 696 142]
 [124 120 664]]
              precision    recall  f1-score   support

      INSIDE       0.66      0.67      0.66       735
IN_VESTIBULE       0.71      0.72      0.71       973
     OUTSIDE       0.75      0.73      0.74       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.71      0.71      0.71      2616
weighted avg       0.71      0.71      0.71      2616

0.7079510703363915


# Пробуем стандартизировать данные используя preprocessing.scale

In [17]:
from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by the scale function.
  
C:\anaconda\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until


# Алгоритм Наивного Байеса после preprocessing.scale

In [18]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train_scaled, y_train).predict(X_test_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[440 127 168]
 [102 722 149]
 [130 131 647]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.60      0.63       735
IN_VESTIBULE       0.74      0.74      0.74       973
     OUTSIDE       0.67      0.71      0.69       908

   micro avg       0.69      0.69      0.69      2616
   macro avg       0.69      0.68      0.69      2616
weighted avg       0.69      0.69      0.69      2616

0.6915137614678899


# Алгоритм Деревьев решений после preprocessing.scale

In [19]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train_scaled, y_train).predict(X_test_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[498 144  93]
 [165 685 123]
 [ 98 144 666]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.68      0.67       735
IN_VESTIBULE       0.70      0.70      0.70       973
     OUTSIDE       0.76      0.73      0.74       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.70      0.71      0.70      2616
weighted avg       0.71      0.71      0.71      2616

0.7068042813455657


# Алгоритм SVM после preprocessing.scale

In [20]:
svm = SVC()
y_pred = svm.fit(X_train_scaled, y_train).predict(X_test_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[515 149  71]
 [111 703 159]
 [122  80 706]]
              precision    recall  f1-score   support

      INSIDE       0.69      0.70      0.69       735
IN_VESTIBULE       0.75      0.72      0.74       973
     OUTSIDE       0.75      0.78      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.73      0.73      0.73      2616
weighted avg       0.74      0.74      0.74      2616

0.735474006116208


# Алгоритм KNN после preprocessing.scale

In [21]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train_scaled, y_train).predict(X_test_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[501 151  83]
 [142 693 138]
 [117 126 665]]
              precision    recall  f1-score   support

      INSIDE       0.66      0.68      0.67       735
IN_VESTIBULE       0.71      0.71      0.71       973
     OUTSIDE       0.75      0.73      0.74       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.71      0.71      0.71      2616
weighted avg       0.71      0.71      0.71      2616

0.7106269113149847


In [115]:
X_train.head()

,direction,rssiOne,rssiTwo
5358,0,-74,-80
425,0,-74,-84
5549,1,-84,-74
9140,0,-92,-80
2177,1,-74,-86


# Пробуем провести дискретизацию признаков 

In [223]:
from sklearn.preprocessing import KBinsDiscretizer  
est = KBinsDiscretizer(n_bins=13, encode='ordinal', strategy='uniform')

# X_train['rssiOne'] = X_train.iloc[:,1].values.reshape(6102,1)
# X_train['rssiTwo'] = X_train.iloc[:,2].values.reshape(6102,1)
# X_test['rssiOne'] = X_test.iloc[:,1].values.reshape(2616,1)
# X_test['rssiTwo'] = X_test.iloc[:,2].values.reshape(2616,1)

# X_train['rssiOne']
est.fit(X_train) 
est.transform(X_train)
# X_train


array([[ 0.,  9.,  6.],
       [ 0.,  9.,  5.],
       [12.,  6.,  8.],
       ...,
       [ 0.,  5.,  3.],
       [ 0.,  5., 10.],
       [ 0.,  8.,  7.]])

In [224]:
Xtst = est.transform(X_test)
Xtst

array([[12.,  3.,  3.],
       [12.,  5.,  8.],
       [ 0.,  8.,  6.],
       ...,
       [12.,  8.,  4.],
       [12.,  6.,  6.],
       [ 0.,  7.,  5.]])

# Алгоритм Наивного Байеса после дискретизации признаков

In [225]:
gnb = GaussianNB()
y_pred = gnb.fit(Xtr, y_train).predict(Xtst)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[441 126 168]
 [105 721 147]
 [127 115 666]]
              precision    recall  f1-score   support

      INSIDE       0.66      0.60      0.63       735
IN_VESTIBULE       0.75      0.74      0.75       973
     OUTSIDE       0.68      0.73      0.71       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.69      0.69      0.69      2616
weighted avg       0.70      0.70      0.70      2616

0.6987767584097859


# Алгоритм Деревьев Решений после дискретизации признаков

In [91]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(Xtr, y_train).predict(Xtst)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[504 134  97]
 [109 694 170]
 [ 80  85 743]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.69      0.71       735
IN_VESTIBULE       0.76      0.71      0.74       973
     OUTSIDE       0.74      0.82      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.74      0.74      0.74      2616

0.7419724770642202


# Алгоритм SVM после дискретизации признаков

In [92]:
svm = SVC()
y_pred = svm.fit(Xtr, y_train).predict(Xtst)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[510 138  87]
 [100 714 159]
 [ 93  88 727]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.69      0.71       735
IN_VESTIBULE       0.76      0.73      0.75       973
     OUTSIDE       0.75      0.80      0.77       908

   micro avg       0.75      0.75      0.75      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.75      0.75      0.75      2616

0.7457951070336392


# Алгоритм KNN после дискретизации признаков

In [93]:
knn = KNeighborsClassifier()
y_pred = knn.fit(Xtr, y_train).predict(Xtst)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[489 154  92]
 [125 693 155]
 [ 90 117 701]]
              precision    recall  f1-score   support

      INSIDE       0.69      0.67      0.68       735
IN_VESTIBULE       0.72      0.71      0.72       973
     OUTSIDE       0.74      0.77      0.76       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.72      0.72      0.72      2616
weighted avg       0.72      0.72      0.72      2616

0.7198012232415902


# Пробуем провести бинаризацию признаков 

In [98]:
from sklearn.preprocessing import Binarizer

transformer = Binarizer()
transformer.fit(X_train, y_train)
X_train_binarizer = scaler.transform(X_train)
X_test_real_scaled = scaler.transform(X_test)

MDLP.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  from sklearn.utils import (
MDLP.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  check_array,


# Алгоритм Наивного Байеса после бинаризации признаков

In [99]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train_binarizer, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[434 134 167]
 [ 91 731 151]
 [129  98 681]]
              precision    recall  f1-score   support

      INSIDE       0.66      0.59      0.62       735
IN_VESTIBULE       0.76      0.75      0.76       973
     OUTSIDE       0.68      0.75      0.71       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.71      0.71      0.70      2616

0.7056574923547401


# Алгоритм Деревьев Решений после бинаризации признаков

In [100]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train_binarizer, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[499 146  90]
 [155 686 132]
 [ 93 129 686]]
              precision    recall  f1-score   support

      INSIDE       0.67      0.68      0.67       735
IN_VESTIBULE       0.71      0.71      0.71       973
     OUTSIDE       0.76      0.76      0.76       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.71      0.71      0.71      2616
weighted avg       0.72      0.72      0.72      2616

0.7152140672782875


# Алгоритм SVM после бинаризации признаков

In [102]:
svm = SVC()
y_pred = svm.fit(X_train_binarizer, y_train).predict(X_test_real_scaled)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[515 149  71]
 [102 713 158]
 [117  74 717]]
              precision    recall  f1-score   support

      INSIDE       0.70      0.70      0.70       735
IN_VESTIBULE       0.76      0.73      0.75       973
     OUTSIDE       0.76      0.79      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.74      0.74      0.74      2616

0.7435015290519877


# Алгоритм KNN после бинаризации признаков

In [103]:
knn = KNeighborsClassifier()
y_pred = knn.fit(Xtr, y_train).predict(Xtst)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[489 154  92]
 [125 693 155]
 [ 90 117 701]]
              precision    recall  f1-score   support

      INSIDE       0.69      0.67      0.68       735
IN_VESTIBULE       0.72      0.71      0.72       973
     OUTSIDE       0.74      0.77      0.76       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.72      0.72      0.72      2616
weighted avg       0.72      0.72      0.72      2616

0.7198012232415902


# Проводим препроцессинг путем применения метода MinMaxScaler

In [108]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

minMaxScaler = MinMaxScaler()
minMaxScaler.fit(X_train, y_train)
X_train_MMScaler = scaler.transform(X_train)
X_test_MMScaler = scaler.transform(X_test)

C:\anaconda\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
MDLP.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  from sklearn.utils import (
MDLP.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  check_array,


# Алгоритм Наивного Байеса после применения MinMaxScaler

In [109]:
# gnb = GaussianNB()
y_pred = gnb.fit(X_train_MMScaler, y_train).predict(X_test_MMScaler)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[434 134 167]
 [ 91 731 151]
 [129  98 681]]
              precision    recall  f1-score   support

      INSIDE       0.66      0.59      0.62       735
IN_VESTIBULE       0.76      0.75      0.76       973
     OUTSIDE       0.68      0.75      0.71       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.71      0.71      0.70      2616

0.7056574923547401


# Алгоритм Деревьев Решений после применения MinMaxScaler

In [110]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train_MMScaler, y_train).predict(X_test_MMScaler)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[499 146  90]
 [155 686 132]
 [ 93 129 686]]
              precision    recall  f1-score   support

      INSIDE       0.67      0.68      0.67       735
IN_VESTIBULE       0.71      0.71      0.71       973
     OUTSIDE       0.76      0.76      0.76       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.71      0.71      0.71      2616
weighted avg       0.72      0.72      0.72      2616

0.7152140672782875


# Алгоритм SVM после применения MinMaxScaler

In [111]:
svm = SVC()
y_pred = svm.fit(X_train_MMScaler, y_train).predict(X_test_MMScaler)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[515 149  71]
 [102 713 158]
 [117  74 717]]
              precision    recall  f1-score   support

      INSIDE       0.70      0.70      0.70       735
IN_VESTIBULE       0.76      0.73      0.75       973
     OUTSIDE       0.76      0.79      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.74      0.74      0.74      2616

0.7435015290519877


# Алгоритм KNN после применения MinMaxScaler

In [112]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train_MMScaler, y_train).predict(X_test_MMScaler)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[494 168  73]
 [142 690 141]
 [106 108 694]]
              precision    recall  f1-score   support

      INSIDE       0.67      0.67      0.67       735
IN_VESTIBULE       0.71      0.71      0.71       973
     OUTSIDE       0.76      0.76      0.76       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.71      0.72      0.71      2616
weighted avg       0.72      0.72      0.72      2616

0.7178899082568807
